In [ ]:
import re
import psycopg2

In [ ]:
def remove_images(text):
    text = re.sub(r'!\[Image\]\S*', '<user sent a image>', text)
    return text

In [ ]:
def remove_urls(text):
    text = re.sub(r'http\S+|www\.\S+', '<user sent a url>', text)
    return text

In [ ]:
def remove_code_blocks(text):
    text = re.sub(r'```(.*?)```', '<user sent a code block>', text, flags=re.DOTALL)
    return text

In [ ]:
def process_reply(text):
    text = re.sub(r'(>.)(.*)', r'<user is replying to: \2 >', text)
    return text

In [ ]:
repo_owner = "microsoft"
repo_name = "vscode"

DATABASE_CONFIG = {
    "dbname": "vscode_data",
    "user": "postgres",
    "password": "",
    "host": "",
    "port": 5432,
}

In [ ]:
conn = psycopg2.connect(**DATABASE_CONFIG)
cursor = conn.cursor()

In [ ]:
cursor.execute(
    """
    SELECT 
        comment_id,
        created_at,
        body
    FROM 
        issue_comments_from_release
    """
)
comments = cursor.fetchall()

In [ ]:
for comment in comments:
    comment_id, created_at, body = comment
    body = remove_code_blocks(body)
    body = remove_images(body)
    body = remove_urls(body)
    body = process_reply(body)
    print(body)
    cursor.execute(
        "UPDATE issue_comments_from_release SET body = %s WHERE comment_id = %s", (body, comment_id)
    )

conn.commit()  